# Address Parser

Is able to recognize named entities and format outputs based on a defined schema

In [1]:


GCP_PROJECT_NAME = "single-azimuth-413609"

In [2]:
!pip install --quiet --upgrade google-cloud-aiplatform 
!pip install --quiet --upgrade google-auth



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
!gcloud auth application-default login --scopes=openid,https://www.googleapis.com/auth/userinfo.email,https://www.googleapis.com/auth/cloud-platform,https://www.googleapis.com/auth/sqlservice.login,https://www.googleapis.com/auth/drive

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=4oCvq3SgCR1jnwxtixGt6ZhLgBjEdr&access_type=offline&code_challenge=02T-ecmi9TvqMRQBMLwo_sVSitQqmN2SprRemd3fwfU&code_challenge_method=S256


Credentials saved to file: [/home/vincent/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "single-azimuth-413609" was added to ADC which can be used by Google client libraries for billing and quota. Note that 

In [4]:
!gcloud config set project {GCP_PROJECT_NAME}

Updated property [core/project].


In [5]:
!gcloud services enable aiplatform.googleapis.com

In [8]:
# Importing the library 
import google.auth

# Setting up credentials
credentials, _ = google.auth.default()
# authed_http = google.auth.transport.requests.AuthorizedSession(credentials)

In [9]:
import vertexai
from vertexai.preview.language_models import TextGenerationModel

vertexai.init(project=GCP_PROJECT_NAME, location="us-central1")
parameters = {
    "candidate_count": 1,
    "max_output_tokens": 126,
    "temperature": 0,
    "top_p": 1
}
model = TextGenerationModel.from_pretrained("text-bison")


## Basic french address

In [10]:
response = model.predict(
    """Extract and split the address parameters from the text below in a JSON format.
be very precise, differentiate the street number, multiplier, street type, street name, zip code, city
provide a rating number to indicate your confidence level. 0 is the lowest confidence, 1 is the maximum.
provide only one response for one address input

Text: 37, rue du champ du pardon, 76000 rouen
JSON: Text: 37, rue du champ du pardon, 76000 rouen
JSON: {
  \"street_number\": \"37\",
  \"street_type\": \"rue\",
  \"street_name\": \"du champ du pardon\",
  \"zip_code\": \"76000\",
  \"city\": \"rouen\",
  \"confidence\": 1
}

Text: 10-82 Cavée St Gervais, 76000 rouen
JSON: Text: 10-82 Cavée St Gervais, 76000 rouen
JSON: {
  \"street_number\": \"10-82\",
  \"street_type\": \"Cavée\",
  \"street_name\": \"St Gervais\",
  \"zip_code\": \"76000\",
  \"city\": \"rouen\",
  \"confidence\": 1
}

Text: 1, rue de la Paix, 75008 Paris
JSON:
""",
    **parameters
)
print(f"Response from Model: {response.text}")

Response from Model:  {
  "street_number": "1",
  "street_type": "rue",
  "street_name": "de la Paix",
  "zip_code": "75008",
  "city": "Paris",
  "confidence": 1
}


### Unusual French address

In [11]:
response = model.predict(
    """Extract and split the address parameters from the text below in a JSON format.
be very precise, differentiate the street number, multiplier, street type, street name, zip code, city
provide a rating number to indicate your confidence level. 0 is the lowest confidence, 1 is the maximum.
provide only one response for one address input

Text: 37, rue du champ du pardon, 76000 rouen
JSON: Text: 37, rue du champ du pardon, 76000 rouen
JSON: {
  \"street_number\": \"37\",
  \"street_type\": \"rue\",
  \"street_name\": \"du champ du pardon\",
  \"zip_code\": \"76000\",
  \"city\": \"rouen\",
  \"confidence\": 1
}

Text: 10-82 Cavée St Gervais, 76000 rouen
JSON: Text: 10-82 Cavée St Gervais, 76000 rouen
JSON: {
  \"street_number\": \"10-82\",
  \"street_type\": \"Cavée\",
  \"street_name\": \"St Gervais\",
  \"zip_code\": \"76000\",
  \"city\": \"rouen\",
  \"confidence\": 1
}

Text: 78 bis Cavee des Ecameaux 76500 Elbeuf
JSON:
""",
    **parameters
)
print(f"Response from Model: {response.text}")

Response from Model:  {
  "street_number": "78",
  "street_multiplier": "bis",
  "street_type": "Cavee",
  "street_name": "des Ecameaux",
  "zip_code": "76500",
  "city": "Elbeuf",
  "confidence": 1
}


In [12]:
response = model.predict(
    """Extract and split the address parameters from the text below in a JSON format.
be very precise, differentiate the street number, multiplier, street type, street name, zip code, city
provide a rating number to indicate your confidence level. 0 is the lowest confidence, 1 is the maximum.
provide only one response for one address input

Text: 37, rue du champ du pardon, 76000 rouen
JSON: Text: 37, rue du champ du pardon, 76000 rouen
JSON: {
  \"street_number\": \"37\",
  \"street_type\": \"rue\",
  \"street_name\": \"du champ du pardon\",
  \"zip_code\": \"76000\",
  \"city\": \"rouen\",
  \"confidence\": 1
}

Text: 78 bis Cavee des Ecameaux 76500 Elbeuf
JSON:
""",
    **parameters
)
print(f"Response from Model: {response.text}")

Response from Model:  {
  "street_number": "78",
  "street_multiplier": "bis",
  "street_type": "Cavee",
  "street_name": "des Ecameaux",
  "zip_code": "76500",
  "city": "Elbeuf",
  "confidence": 1
}


## Complex US address

In [13]:
response = model.predict(
    """Extract and split the address parameters from the text below in a JSON format.
be very precise, differentiate the street number, multiplier, street type, street name, zip code, city
provide a rating number to indicate your confidence level. 0 is the lowest confidence, 1 is the maximum.
provide only one response for one address input

Text: 37, rue du champ du pardon, 76000 rouen
JSON: Text: 37, rue du champ du pardon, 76000 rouen
JSON: {
  \"street_number\": \"37\",
  \"street_type\": \"rue\",
  \"street_name\": \"du champ du pardon\",
  \"zip_code\": \"76000\",
  \"city\": \"rouen\",
  \"confidence\": 1
}

Text: 10-82 Cavée St Gervais, 76000 rouen
JSON: Text: 10-82 Cavée St Gervais, 76000 rouen
JSON: {
  \"street_number\": \"10-82\",
  \"street_type\": \"Cavée\",
  \"street_name\": \"St Gervais\",
  \"zip_code\": \"76000\",
  \"city\": \"rouen\",
  \"confidence\": 1
}

Text: C/o John Doe LLC, 111, 8th Ave Ste 1509, Oklahoma, 74136-1922, USA
JSON:
""",
    **parameters
)
print(f"Response from Model: {response.text}")

Response from Model:  {
  "street_number": "111",
  "street_multiplier": "8th",
  "street_type": "Ave",
  "street_name": "Ste 1509",
  "zip_code": "74136-1922",
  "city": "Oklahoma",
  "country": "USA",
  "confidence": 1
}


## German address 

generator at: https://tarjeta-credito.net/cvv/address-gen-germany/

In [14]:
response = model.predict(
    """Extract and split the address parameters from the text below in a JSON format.
be very precise, differentiate the street number, multiplier, street type, street name, zip code, city
provide a rating number to indicate your confidence level. 0 is the lowest confidence, 1 is the maximum.
provide only one response for one address input

Text: 37, rue du champ du pardon, 76000 rouen
JSON: Text: 37, rue du champ du pardon, 76000 rouen
JSON: {
  \"street_number\": \"37\",
  \"street_type\": \"rue\",
  \"street_name\": \"du champ du pardon\",
  \"zip_code\": \"76000\",
  \"city\": \"rouen\",
  \"confidence\": 1
}

Text: 10-82 Cavée St Gervais, 76000 rouen
JSON: Text: 10-82 Cavée St Gervais, 76000 rouen
JSON: {
  \"street_number\": \"10-82\",
  \"street_type\": \"Cavée\",
  \"street_name\": \"St Gervais\",
  \"zip_code\": \"76000\",
  \"city\": \"rouen\",
  \"confidence\": 1
}

Text: Haßfurter Str. 1, 91056 Erlangen, Germany.
JSON:
""",
    **parameters
)
print(f"Response from Model: {response.text}")

Response from Model:  {
  "street_number": "1",
  "street_type": "Str.",
  "street_name": "Haßfurter",
  "zip_code": "91056",
  "city": "Erlangen",
  "country": "Germany",
  "confidence": 1
}


### german address with Country not mentionned

In [15]:
response = model.predict(
    """Extract and split the address parameters from the text below in a JSON format.
be very precise, differentiate the street number, multiplier, street type, street name, zip code, city and country
provide a rating number to indicate your confidence level. 0 is the lowest confidence, 1 is the maximum.
provide only one response for one address input

Text: 37, rue du champ du pardon, 76000 rouen
JSON: Text: 37, rue du champ du pardon, 76000 rouen
JSON: {
  \"street_number\": \"37\",
  \"street_type\": \"rue\",
  \"street_name\": \"du champ du pardon\",
  \"zip_code\": \"76000\",
  \"city\": \"rouen\",
  \"country\": \"France\",
  \"confidence\": 1
}

Text: 10-82 Cavée St Gervais, 76000 rouen
JSON: Text: 10-82 Cavée St Gervais, 76000 rouen
JSON: {
  \"street_number\": \"10-82\",
  \"street_type\": \"Cavée\",
  \"street_name\": \"St Gervais\",
  \"zip_code\": \"76000\",
  \"city\": \"rouen\",
  \"country\": \"France\",
  \"confidence\": 1
}

Text: Auschnippe 9, 37170 Uslar
JSON:
""",
    **parameters
)
print(f"Response from Model: {response.text}")

Response from Model:  {
  "street_number": "9",
  "street_name": "Auschnippe",
  "zip_code": "37170",
  "city": "Uslar",
  "country": "Germany",
  "confidence": 1
}


## Confidence level w/ False address

In [16]:
response = model.predict(
    """Extract and split the address parameters from the text below in a JSON format.
be very precise, differentiate the street number, multiplier, street type, street name, zip code, city
provide a rating number to indicate your confidence level. 0 is the lowest confidence, 1 is the maximum.
provide only one response for one address input

Text: 37, rue du champ du pardon, 76000 rouen
JSON: 
{
  \"street_number\": \"37\",
  \"street_type\": \"rue\",
  \"street_name\": \"du champ du pardon\",
  \"zip_code\": \"76000\",
  \"city\": \"rouen\",
  \"confidence\": 1
}

Text: 10-82 Cavée St Gervais, 76000 rouen
JSON: 
{
  \"street_number\": \"10-82\",
  \"street_type\": \"Cavée\",
  \"street_name\": \"St Gervais\",
  \"zip_code\": \"76000\",
  \"city\": \"rouen\",
  \"confidence\": 1
}

Text: ici c'est paris !, 75001 Paris
JSON:

""",
    **parameters
)
print(f"{response.text}")

 {
  "street_number": null,
  "street_type": null,
  "street_name": null,
  "zip_code": "75001",
  "city": "Paris",
  "confidence": 0.5
}


### Confidence level w/ False address and Country not mentioned

In [17]:
response = model.predict(
    """Extract and split the address parameters from the text below in a JSON format.
be very precise, differentiate the street number, multiplier, street type, street name, zip code, city
provide a rating number to indicate your confidence level. 0 is the lowest confidence, 1 is the maximum.
provide only one response for one address input

Text: 37, rue du champ du pardon, 76000 rouen
JSON: 
{
  \"street_number\": \"37\",
  \"street_type\": \"rue\",
  \"street_name\": \"du champ du pardon\",
  \"zip_code\": \"76000\",
  \"city\": \"rouen\",
  \"country\": \"France\",
  \"confidence\": 1
}

Text: 10-82 Cavée St Gervais, 76000 rouen
JSON: 
{
  \"street_number\": \"10-82\",
  \"street_type\": \"Cavée\",
  \"street_name\": \"St Gervais\",
  \"zip_code\": \"76000\",
  \"city\": \"rouen\",
  \"country\": \"France\",
  \"confidence\": 1
}

Text: 452 quater grande rue general 85000 le palais vendee
JSON:

""",
    **parameters
)
print(f"{response.text}")


 {
  "street_number": "452",
  "street_multiplier": "quater",
  "street_type": "grande rue",
  "street_name": "general",
  "zip_code": "85000",
  "city": "le palais",
  "department": "vendee",
  "country": "France",
  "confidence": 1
}


## Programmatic

Using Langchain we can have a more programmatic way 

In [110]:
!pip install --upgrade --quiet  langchain-core langchain-google-vertexai


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [112]:
from langchain_google_vertexai import VertexAI

# candidate count parameter missing ?
model = VertexAI(model_name="text-bison", top_p=1, temperature=0, max_output_tokens=126)

In [120]:

from langchain_core.pydantic_v1 import BaseModel
from typing import Optional

from langchain_core.output_parsers import JsonOutputParser


class Address(BaseModel):
    street_number: Optional[str]
    street_multiplier: Optional[str]
    street_type: Optional[str]
    street_name: Optional[str]
    locality: Optional[str]  #lieu-dit
    zip_code: Optional[str]
    city: Optional[str]
    country: Optional[str]
    confidence: int


parser = JsonOutputParser(pydantic_object=Address)
# pprint(parser.get_format_instructions())


In [118]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts import PromptTemplate

examples = [
    {
        "address": "37, rue du champ du pardon, 76000 rouen",
        "answer": """{{
  \"street_number\": \"37\",
  \"street_type\": \"rue\",
  \"street_name\": \"du champ du pardon\",
  \"zip_code\": \"76000\",
  \"city\": \"rouen\",
  \"country\": \"France\",
  \"confidence\": 1
}}"""
    },
    {
        "address": "10-82 Cavée St Gervais, hameau de charette 76000 rouen",
        "answer": """{{
  \"street_number\": \"10-82\",
  \"street_type\": \"Cavée\",
  \"street_name\": \"St Gervais\",
  \"locality\": \"hameau de charette\",
  \"zip_code\": \"76000\",
  \"city\": \"rouen\",
  \"country\": \"France\",
  \"confidence\": 1
}}"""
    },
]

prefix = """Extract and split the address parameters from the text below in a JSON format.
be very precise, differentiate the street number, multiplier, street type, street name, zip code, city
provide a rating number to indicate your confidence level. 0 is the lowest confidence, 1 is the maximum.
provide only one response for one address input"""

example_prompt = PromptTemplate(
    template="""Text: {address}\nJSON: {answer}""",
    # template="""Answer the user query.\n{format_instructions}\nText: {address}\nJSON: {answer}\n""",
    input_variables=["address", "answer"],
    # partial_variables={"format_instructions": parser.get_format_instructions()}
)

# print(example_prompt.format(**examples[0]))

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix="Text: {address}\nJSON:",
    input_variables=["address"],

)

# print(prompt.format(address="78 bis Cavee des Ecameaux 76500 Elbeuf"))


chain = prompt | model | parser

### Testing

In [121]:

address = "1 bis rue de la Paix, 75008 Paris"
Address(**chain.invoke({"address": address}))


Address(street_number='1', street_multiplier='bis', street_type='rue', street_name='de la Paix', locality=None, zip_code='75008', city='Paris', country='France', confidence=1)

In [122]:

address = "Auschnippe 9, 37170 Uslar"
Address(**chain.invoke({"address": address}))


Address(street_number='9', street_multiplier=None, street_type=None, street_name='Auschnippe', locality=None, zip_code='37170', city='Uslar', country='Germany', confidence=1)

In [24]:

address = "C/o John Doe LLC, 111, 8th Ave Ste 1509, Oklahoma, 74136-1922, USA"
add = Address(**chain.invoke({"address": address}))
add


Address(street_number='111', street_multiplier='8th', street_type='Ave', street_name='Ste 1509', zip_code='74136-1922', city='Oklahoma', country='USA', confidence=1)

### Serialize / Compress

In [25]:
import sys
from pprint import pprint
import zlib

# size of object in bytes
pprint(sys.getsizeof(add))
pprint(sys.getsizeof(add.json()))
pprint(sys.getsizeof(zlib.compress(add.json().encode())))

56
231
164


## Auto-Testing

pre-requisite: Install dependencies: pandas, pyarrow, etc...

Use of 'base nationale des Adresses' for French addresses (BAN)

#### Exploratory Data Analysis & Cleanup

In [327]:
import pandas as pd
from pprint import pprint
import numpy as np

df_addresses = pd.read_csv('./dataset/adresses-01.csv', delimiter=';', low_memory=False)
df_addresses.shape

(265542, 23)

In [328]:
df_addresses.head()

,id,id_fantoir,numero,rep,nom_voie,code_postal,code_insee,nom_commune,code_insee_ancienne_commune,nom_ancienne_commune,...,lat,type_position,alias,nom_ld,libelle_acheminement,nom_afnor,source_position,source_nom_voie,certification_commune,cad_parcelles
0,01001_4b50r5_00630,NaN,630,NaN,la Chèvre,1400,1001,L'Abergement-Clémenciat,NaN,NaN,...,46.132481,segment,NaN,NaN,L'ABERGEMENT-CLEMENCIAT,LA CHEVRE,inconnue,inconnue,0,NaN
1,01001_g0ru02_00108,NaN,108,NaN,Clemencia,1400,1001,L'Abergement-Clémenciat,NaN,NaN,...,46.133867,parcelle,NaN,NaN,L'ABERGEMENT-CLEMENCIAT,CLEMENCIA,cadastre,cadastre,0,01001000ZL0263
2,01001_ngzlqw_00009,NaN,9,NaN,Imp des Epis,1400,1001,L'Abergement-Clémenciat,NaN,NaN,...,46.152646,NaN,NaN,NaN,L'ABERGEMENT-CLEMENCIAT,IMP DES EPIS,arcep,arcep,0,NaN
3,01001_ngzlqw_00023,NaN,23,NaN,Imp des Epis,1400,1001,L'Abergement-Clémenciat,NaN,NaN,...,46.152646,NaN,NaN,NaN,L'ABERGEMENT-CLEMENCIAT,IMP DES EPIS,arcep,arcep,0,NaN
4,01001_ngzlqw_00026,NaN,26,NaN,Imp des Epis,1400,1001,L'Abergement-Clémenciat,NaN,NaN,...,46.152646,NaN,NaN,NaN,L'ABERGEMENT-CLEMENCIAT,IMP DES EPIS,arcep,arcep,0,NaN


In [329]:
df_addresses.drop(
    columns=['id_fantoir', 'code_insee', 'code_insee_ancienne_commune', 'nom_ancienne_commune', 'x', 'y', 'lon', 'lat',
             'type_position', 'source_position', 'source_nom_voie', 'certification_commune', 'cad_parcelles'],
    inplace=True)

In [330]:
df_addresses.replace(np.nan, '', regex=True, inplace=True)
df_addresses.replace('_1', '', regex=True, inplace=True)


In [331]:
df_addresses.drop_duplicates(['nom_voie', 'nom_commune', 'nom_ld'], inplace=True)
df_addresses.head(100)

,id,numero,rep,nom_voie,code_postal,nom_commune,alias,nom_ld,libelle_acheminement,nom_afnor
0,01001_4b50r5_00630,630,,la Chèvre,1400,L'Abergement-Clémenciat,,,L'ABERGEMENT-CLEMENCIAT,LA CHEVRE
1,01001_g0ru02_00108,108,,Clemencia,1400,L'Abergement-Clémenciat,,,L'ABERGEMENT-CLEMENCIAT,CLEMENCIA
2,01001_ngzlqw_00009,9,,Imp des Epis,1400,L'Abergement-Clémenciat,,,L'ABERGEMENT-CLEMENCIAT,IMP DES EPIS
12,01001_0165_00019,19,,Route de la Fontaine,1400,L'Abergement-Clémenciat,,,L'ABERGEMENT-CLEMENCIAT,ROUTE DE LA FONTAINE
44,01001_0115_00095,95,,Route de Clémenciat,1400,L'Abergement-Clémenciat,,,L'ABERGEMENT-CLEMENCIAT,ROUTE DE CLEMENCIAT
...,...,...,...,...,...,...,...,...,...,...
626,01002_0180_00065,65,,Route de l’Oiselon,1640,L'Abergement-de-Varey,,,ABERGEMENT-DE-VAREY (L ),ROUTE DE L OISELON
631,01002_0043_00011,11,,Allee du Chateau,1640,L'Abergement-de-Varey,,,ABERGEMENT-DE-VAREY (L ),ALLEE DU CHATEAU
634,01002_0049_01053,1053,,Route de la Cordiere,1640,L'Abergement-de-Varey,,,ABERGEMENT-DE-VAREY (L ),ROUTE DE LA CORDIERE
637,01002_0250_00087,87,,Chemin de sous la Tour,1640,L'Abergement-de-Varey,,,ABERGEMENT-DE-VAREY (L ),CHEMIN DE SOUS LA TOUR


In [30]:
def dataframe_row_to_pydantic_object(row: pd.Series) -> Address:
    """
    Converts a row from a Pandas DataFrame to a Pydantic object.
    Args:
        row (pd.Series): A single row from the DataFrame.
    Returns:
        SomeModel: A Pydantic object with the data from the row.
    """
    #missing street_type
    return Address(street_number=str(row['numero']), street_multiplier=row['rep'], street_name=row['nom_voie'],
                   zip_code=str(row['code_postal']), city=row['nom_commune'], confidence=1)


In [106]:
dataframe_row_to_pydantic_object(df_addresses.iloc[0])

Address(street_number='630', street_multiplier='', street_type=None, street_name='la Chèvre', zip_code='1400', city="L'Abergement-Clémenciat", country=None, confidence=1)

In [191]:
def full_address_generator(row: pd.Series) -> str:
    """
    Generates a one line address based on all the available fields from BAN
    :param row: pd.Series of one address from BAN
    :return: 
    """
    if row['nom_ld'] != '':
        row['nom_ld'] = ', ' + row['nom_ld'] #todo here we are cheating a little bit, in order to differentiate the locality (lieu-dit) we insert a comma, actually otherwise even for a human it would be very difficult to differentiate ! 
    strings = [str(row['numero']), str(row['rep']), str(row['nom_voie']), str(row['nom_ld']), str(row['code_postal']),
               str(row['nom_commune'])]
    return ' '.join(filter(None, strings))
    # return f"{row['numero']}{('',' '+str(row['rep']))[row['rep'] == 'nan']}{' ' + row['nom_voie']}{('', ', '+str(row['nom_ld']))[row['nom_ld'] == 'nan']}{' '+str(row['code_postal'])}{' ' + row['nom_commune']}"

In [333]:

index = 4568
pprint(df_addresses.iloc[index])
full_address_generator(df_addresses.iloc[index])

id                               01378_0019_00001
numero                                          1
rep                                              
nom_voie                    Route de la Charriere
code_postal                                  1800
nom_commune             Saint-Maurice-de-Gourdans
alias                                            
nom_ld                                           
libelle_acheminement    SAINT-MAURICE-DE-GOURDANS
nom_afnor                   ROUTE DE LA CHARRIERE
Name: 47639, dtype: object


'1 Route de la Charriere 1800 Saint-Maurice-de-Gourdans'

Add one line address to Dataframe

In [334]:
df_addresses['full_add'] = df_addresses.apply(lambda _row: full_address_generator(_row), axis=1)

In [335]:
df_addresses.head(50)

,id,numero,rep,nom_voie,code_postal,nom_commune,alias,nom_ld,libelle_acheminement,nom_afnor,full_add
0,01001_4b50r5_00630,630,,la Chèvre,1400,L'Abergement-Clémenciat,,,L'ABERGEMENT-CLEMENCIAT,LA CHEVRE,630 la Chèvre 1400 L'Abergement-Clémenciat
1,01001_g0ru02_00108,108,,Clemencia,1400,L'Abergement-Clémenciat,,,L'ABERGEMENT-CLEMENCIAT,CLEMENCIA,108 Clemencia 1400 L'Abergement-Clémenciat
2,01001_ngzlqw_00009,9,,Imp des Epis,1400,L'Abergement-Clémenciat,,,L'ABERGEMENT-CLEMENCIAT,IMP DES EPIS,9 Imp des Epis 1400 L'Abergement-Clémenciat
12,01001_0165_00019,19,,Route de la Fontaine,1400,L'Abergement-Clémenciat,,,L'ABERGEMENT-CLEMENCIAT,ROUTE DE LA FONTAINE,19 Route de la Fontaine 1400 L'Abergement-Clém...
44,01001_0115_00095,95,,Route de Clémenciat,1400,L'Abergement-Clémenciat,,,L'ABERGEMENT-CLEMENCIAT,ROUTE DE CLEMENCIAT,95 Route de Clémenciat 1400 L'Abergement-Cléme...
60,01001_0370_00036,36,,Impasse des Soyeux,1400,L'Abergement-Clémenciat,,,L'ABERGEMENT-CLEMENCIAT,IMPASSE DES SOYEUX,36 Impasse des Soyeux 1400 L'Abergement-Clémen...
67,01001_0250_00024,24,,Impasse des Merles,1400,L'Abergement-Clémenciat,,,L'ABERGEMENT-CLEMENCIAT,IMPASSE DES MERLES,24 Impasse des Merles 1400 L'Abergement-Clémen...
79,01001_0175_00072,72,,Route des Garaudes,1400,L'Abergement-Clémenciat,,,L'ABERGEMENT-CLEMENCIAT,ROUTE DES GARAUDES,72 Route des Garaudes 1400 L'Abergement-Clémen...
109,01001_0395_00009,9,,Impasse des Vers à Soie,1400,L'Abergement-Clémenciat,,,L'ABERGEMENT-CLEMENCIAT,IMPASSE DES VERS A SOIE,9 Impasse des Vers à Soie 1400 L'Abergement-Cl...
114,01001_0290_00005,5,,Chemin de l'Ordre,1400,L'Abergement-Clémenciat,,,L'ABERGEMENT-CLEMENCIAT,CHEMIN DE L ORDRE,5 Chemin de l'Ordre 1400 L'Abergement-Clémenciat


### Testing

In [338]:
def score_ban_from_split(split: Address, ban: pd.Series, _print: bool = False) -> tuple[int, int]:
    """
    Defines a score to compare an Address object and a pd.Series of BAN
    :param split: Address object
    :param ban: pd.Series row of BAN Dataframe
    :param _print: print the input objects (debug purposes)
    :return: score from 0 to 6 and _score one hot encoded from 0 to 111111
    """
    if _print: 
        print(split)
        print(ban)
    _score = 0
    if split.street_number == str(ban['numero']): _score += 1
    if (not (bool(split.street_multiplier) | bool(ban['rep']))) | (
            split.street_multiplier == str(ban['rep'])): _score += 10
    if (split.street_type + ' ' + split.street_name) == str(ban['nom_voie']): _score += 100
    if (not (bool(split.locality) | bool(ban['nom_ld']))) | (split.locality == ban['nom_ld']): _score += 1000
    if split.zip_code == str(ban['code_postal']): _score += 10000
    if split.city == str(ban['nom_commune']): _score += 100000
    return str(_score).count('1'), _score


In [339]:
row = df_addresses.iloc[6892]
split_adr = Address(**chain.invoke({"address": row['full_add']}))
score = score_ban_from_split(split_adr, row, True)
score

street_number='31' street_multiplier=None street_type='Rue' street_name='des Tilleuls' locality=None zip_code='1000' city='Saint-Denis-lès-Bourg' country='France' confidence=1
id                                                    01344_0469_00031
numero                                                              31
rep                                                                   
nom_voie                                              Rue des Tilleuls
code_postal                                                       1000
nom_commune                                      Saint-Denis-lès-Bourg
alias                                                                 
nom_ld                                                                
libelle_acheminement                             SAINT-DENIS-LES-BOURG
nom_afnor                                             RUE DES TILLEULS
full_add                31 Rue des Tilleuls 1000 Saint-Denis-lès-Bourg
Name: 71365, dtype: object


(6, 111111)

In [340]:
row = df_addresses.iloc[4564]
split_adr = Address(**chain.invoke({"address": row['full_add']}))
score = score_ban_from_split(split_adr, row, True)
score

street_number='2' street_multiplier='7' street_type='Chemin' street_name='de la Riviere' locality=None zip_code='1800' city='Saint-Maurice-de-Gourdans' country='France' confidence=1
id                                                     01378_cful37_00002
numero                                                                  2
rep                                                                      
nom_voie                                           7 Chemin de la Riviere
code_postal                                                          1800
nom_commune                                     Saint-Maurice-de-Gourdans
alias                                                                    
nom_ld                                                                   
libelle_acheminement                            SAINT-MAURICE-DE-GOURDANS
nom_afnor                                          7 CHEMIN DE LA RIVIERE
full_add                2 7 Chemin de la Riviere 1800 Saint-Maurice-de...
Name

(4, 111001)

In [341]:
row = df_addresses.iloc[5521]
split_adr = Address(**chain.invoke({"address": row['full_add']}))
score = score_ban_from_split(split_adr, row, True)
score

street_number='2' street_multiplier=None street_type='Impasse' street_name='Bouly' locality=None zip_code='1290' city='Saint-Jean-sur-Veyle' country='France' confidence=1
id                                               01365_0041_00002
numero                                                          2
rep                                                              
nom_voie                                            Impasse Bouly
code_postal                                                  1290
nom_commune                                  Saint-Jean-sur-Veyle
alias                                                            
nom_ld                                                           
libelle_acheminement                         SAINT-JEAN-SUR-VEYLE
nom_afnor                                           IMPASSE BOULY
full_add                2 Impasse Bouly 1290 Saint-Jean-sur-Veyle
Name: 56694, dtype: object


(6, 111111)

In [343]:
row = df_addresses.iloc[11123]
split_adr = Address(**chain.invoke({"address": row['full_add']}))
score = score_ban_from_split(split_adr, row, True)
score

street_number='206' street_multiplier=None street_type='Route' street_name='de Jailleux' locality=None zip_code='1120' city='Montluel' country='France' confidence=1
id                                         01262_0321_00206
numero                                                  206
rep                                                        
nom_voie                                  Route de Jailleux
code_postal                                            1120
nom_commune                                        Montluel
alias                                                      
nom_ld                                                     
libelle_acheminement                               MONTLUEL
nom_afnor                                 ROUTE DE JAILLEUX
full_add                206 Route de Jailleux 1120 Montluel
Name: 116029, dtype: object


(6, 111111)

In [344]:
row = df_addresses.iloc[5546]
split_adr = Address(**chain.invoke({"address": row['full_add']}))
score = score_ban_from_split(split_adr, row, 1)
score

street_number='39' street_multiplier=None street_type='Chemin' street_name='des Bugnes' locality=None zip_code='1290' city='Saint-Jean-sur-Veyle' country='France' confidence=1
id                                                    01365_0056_00039
numero                                                              39
rep                                                                   
nom_voie                                             Chemin des Bugnes
code_postal                                                       1290
nom_commune                                       Saint-Jean-sur-Veyle
alias                                                                 
nom_ld                                                                
libelle_acheminement                              SAINT-JEAN-SUR-VEYLE
nom_afnor                                            CHEMIN DES BUGNES
full_add                39 Chemin des Bugnes 1290 Saint-Jean-sur-Veyle
Name: 56927, dtype: object


(6, 111111)

### Score Compute

We need to take a portion of the data set in order to minimize the cost

In [345]:
df_addresses.shape

(25821, 11)

In [350]:
df_test = df_addresses.sample(100).copy()
df_test.shape

(100, 11)

In [312]:
def score_row(_row : pd.Series) -> tuple[int, int]:
    _split_adr = Address(**chain.invoke({"address": row['full_add']}))
    return score_ban_from_split(_split_adr, row)

In [351]:
df_test['score'] = df_test.apply(lambda _row: score_row(_row), axis=1)

In [353]:
df_test.head(100)

,id,numero,rep,nom_voie,code_postal,nom_commune,alias,nom_ld,libelle_acheminement,nom_afnor,full_add,score
8548,01454_0011_00010,10,,Avenue du Bugey,1300,Virignin,,,VIRIGNIN,AVENUE DU BUGEY,10 Avenue du Bugey 1300 Virignin,"(6, 111111)"
225138,01051_0006_00005,5,,Chemin de l'Amont,1450,Bolozon,,,BOLOZON,CHEMIN DE L AMONT,5 Chemin de l'Amont 1450 Bolozon,"(6, 111111)"
258012,01024_0350_00028,28,,Chemin du Colombier,1340,Attignat,,,ATTIGNAT,CHEMIN DU COLOMBIER,28 Chemin du Colombier 1340 Attignat,"(6, 111111)"
3162,01004_0246_00001,1,,Rue de la Commune 1871,1500,Ambérieu-en-Bugey,,,AMBERIEU-EN-BUGEY,RUE DE LA COMMUNE 1871,1 Rue de la Commune 1871 1500 Ambérieu-en-Bugey,"(6, 111111)"
253522,01027_y23xbp_00022,22,,Impasse des Chasseurs,1360,Balan,,,BALAN,IMPASSE DES CHASSEURS,22 Impasse des Chasseurs 1360 Balan,"(6, 111111)"
...,...,...,...,...,...,...,...,...,...,...,...,...
111849,01275_0280_00004,4,,Allee des Ecureuils,1700,Neyron,,,NEYRON,ALLEE DES ECUREUILS,4 Allee des Ecureuils 1700 Neyron,"(6, 111111)"
143137,01224_0042_00017,17,,Rue des Pervenches,1360,Loyettes,,,LOYETTES,RUE DES PERVENCHES,17 Rue des Pervenches 1360 Loyettes,"(6, 111111)"
140612,01232_0150_00210,210,,Montée de la Cote,1851,Marboz,,,MARBOZ,MONTEE DE LA COTE,210 Montée de la Cote 1851 Marboz,"(6, 111111)"
192134,01128_0080_00572,572,,Route de la Ferme de la Foret,1560,Courtes,,,COURTES,ROUTE DE LA FERME DE LA FORET,572 Route de la Ferme de la Foret 1560 Courtes,"(6, 111111)"
